# **Задача 1.**
Поиграемся с датасетом про диабет
Целевая переменная тут явно обозначена как Class variable (очевидно, есть у человека диабет или нет)

Описание датасета:
1. Number of times pregnant - кол-во беременностей
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test - Концентрация глюкозы в плазме через 2 часа при пероральном тесте на толерантность к глюкозе
3. Diastolic blood pressure (mm Hg) - Диастолическое артериальное давление
4. Triceps skin fold thickness (mm)	- Толщина кожной складки трицепса
5. 2-Hour serum insulin (mu U/ml)	- 2-Сывороточный инсулин почасово
6. Body mass index (weight in kg/(height in m)^2) - Индекс массы тела
7. Diabetes pedigree function - есть ли в роду диабет
8. Age (years) - возраст
9. Class variable (0 or 1) - переменная класса

# 1. Импортируем нужные библиотеки

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt

# 2. Загружаем наш датасет

In [2]:
diabet_df = pd.read_csv('Diabetes Binary Classification.csv')
diabet_df.head()

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable (0 or 1)
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# 3. Оценка данных

In [3]:
# выведем размерность нашего датасета:
diabet_df.shape

(768, 9)

In [4]:
# выведем типы данных:
diabet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Number of times pregnant                                                  768 non-null    int64  
 1   Plasma glucose concentration a 2 hours in an oral glucose tolerance test  768 non-null    int64  
 2   Diastolic blood pressure (mm Hg)                                          768 non-null    int64  
 3   Triceps skin fold thickness (mm)                                          768 non-null    int64  
 4   2-Hour serum insulin (mu U/ml)                                            768 non-null    int64  
 5   Body mass index (weight in kg/(height in m)^2)                            768 non-null    float64
 6   Diabetes pedigree function                                         

In [5]:
# выведем основные статистические показатели:
diabet_df.describe().round(3)

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable (0 or 1)
count,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000
mean,3.845,120.895,69.105,20.536,79.799,31.993,0.472,33.241,0.349
std,3.370,31.973,19.356,15.952,115.244,7.884,0.331,11.760,0.477
min,0.000,0.000,0.000,0.000,0.000,0.000,0.078,21.000,0.000
25%,1.000,99.000,62.000,0.000,0.000,27.300,0.244,24.000,0.000
50%,3.000,117.000,72.000,23.000,30.500,32.000,0.372,29.000,0.000
75%,6.000,140.250,80.000,32.000,127.250,36.600,0.626,41.000,1.000
max,17.000,199.000,122.000,99.000,846.000,67.100,2.420,81.000,1.000


In [6]:
diabet_df.rename(columns={'Class variable (0 or 1)': 'Class_variable'}, inplace=True)

# 4. Построим модель Логистической регрессии

In [7]:
X = diabet_df.drop('Class_variable', axis=1)
y = diabet_df.Class_variable
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
# Проверим распределение классов:
y.value_counts()

0    500
1    268
Name: Class_variable, dtype: int64

In [9]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.91      0.79      0.84       437
           1       0.53      0.76      0.62       139

    accuracy                           0.78       576
   macro avg       0.72      0.77      0.73       576
weighted avg       0.82      0.78      0.79       576
               precision    recall  f1-score   support

           0       0.88      0.79      0.83       136
           1       0.59      0.73      0.65        56

    accuracy                           0.77       192
   macro avg       0.73      0.76      0.74       192
weighted avg       0.79      0.77      0.78       192



In [10]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.90      0.79      0.84       432
           1       0.55      0.75      0.63       144

    accuracy                           0.78       576
   macro avg       0.73      0.77      0.74       576
weighted avg       0.81      0.78      0.79       576
               precision    recall  f1-score   support

           0       0.86      0.79      0.82       133
           1       0.60      0.71      0.65        59

    accuracy                           0.77       192
   macro avg       0.73      0.75      0.74       192
weighted avg       0.78      0.77      0.77       192

